**Igor Rudolf**

__Rozwiązanie pracy domowej nr 5__

## 1. Przygotowanie danych

1. Wykonaj `import` potrzebnych pakietów (oprócz poniżej wymienionych), tak aby umieszczony w tym notebooku kod działał.

In [1]:
import time
import sqlite3
import pickle
import numpy as np
import pandas as pd

2. Wczytaj ramki danych, na których będziesz dalej pracował

**UWAGA:**
* Pliki muszą znajdować się w katalogu "travel_stackexchange_com" w tym samym katalogu co ten notebook (plik .ipynb).
* Nazwy tabel muszą być *zgodne z instrukcją zamieszczoną w treści pracy domowej*.


In [2]:
# DOPISZ TU ODPOWIEDNI KOD
Comments = pd.read_csv("travel_stackexchange_com/Comments.csv.gz",
compression = 'gzip')

Posts= pd.read_csv("travel_stackexchange_com/Posts.csv.gz",
compression = 'gzip')

Users= pd.read_csv("travel_stackexchange_com/Users.csv.gz",
compression = 'gzip')

3. Przygotuj bazę danych wykonując poniższą komórkę.

In [3]:
# NIE MODYFIKUJ TEJ KOMÓRKI
SCIEZKA_BAZY = './pd5_baza.db'  # Ścieżka do pliku z bazą danych ('./' oznacza bieżący katalog, czyli będzie to plik w tym samym katalogu, co ten notebook).
with sqlite3.connect(SCIEZKA_BAZY) as conn: # połączenie do bazy danych
    # wewnątrz bloku `with` mamy dostępny obiekt połączenia, które jest automatycznie zamykane po jego opuszczeniu.
    Comments.to_sql("Comments", conn, if_exists='replace')  # jeżeli ramka danych już istnieje, to jest nadpisywana.
    Posts.to_sql("Posts", conn, if_exists='replace')
    Users.to_sql("Users", conn, if_exists='replace')

## 2. Wyniki zapytań SQL

Wykonaj zapytania sql. Poniższy kod zapisze też wynik do pliku bazy - potem można go z niej odczytać.

**Uwaga**: Zapytania powinny się wykonywać nie dłużej niż kilka sekund każde,
jednak czasem występują problemy zależne od systemu, np. pod Linuxem zapytania 3 i 5 potrafią zająć
odp. kilka minut i ponad godzinę. Żeby obejść ten problem pod koniec tej sekcji pokazane jest, jak
zapisać wyniki zapytań do tymczasowych plików `pickle`.


In [ ]:
# NIE MODYFIKUJ TEJ KOMÓRKI (poza ew. zakomentowaniem wywoływania zapytań)
zapytanie_1 = """
SELECT Location, SUM(UpVotes) as TotalUpVotes
FROM Users
WHERE Location != ''
GROUP BY Location
ORDER BY TotalUpVotes DESC
LIMIT 10
"""

zapytanie_2 = """
SELECT STRFTIME('%Y', CreationDate) AS Year,
       STRFTIME('%m', CreationDate) AS Month,
       COUNT(*) AS PostsNumber, MAX(Score) AS MaxScore
FROM Posts
WHERE PostTypeId IN (1, 2)
GROUP BY Year, Month
HAVING PostsNumber > 1000
"""

zapytanie_3 = """
SELECT Id, DisplayName, TotalViews
FROM (
        SELECT OwnerUserId, SUM(ViewCount) as TotalViews
        FROM Posts
        WHERE PostTypeId = 1
        GROUP BY OwnerUserId
    ) AS Questions
JOIN Users
ON Users.Id = Questions.OwnerUserId
ORDER BY TotalViews DESC
LIMIT 10
"""

zapytanie_4 = """
SELECT DisplayName, QuestionsNumber, AnswersNumber, Location, Reputation, UpVotes, DownVotes
FROM (
        SELECT *
        FROM (
                SELECT COUNT(*) as AnswersNumber, OwnerUserId
                FROM Posts
                WHERE PostTypeId = 2
                GROUP BY OwnerUserId
            ) AS Answers
        JOIN
            (
                SELECT COUNT(*) as QuestionsNumber, OwnerUserId
                FROM Posts
                WHERE PostTypeId = 1
                GROUP BY OwnerUserId
            ) AS Questions
        ON Answers.OwnerUserId = Questions.OwnerUserId
        WHERE AnswersNumber > QuestionsNumber
        ORDER BY AnswersNumber DESC
        LIMIT 5
    ) AS PostsCounts
JOIN Users
ON PostsCounts.OwnerUserId = Users.Id
"""

zapytanie_5 = """
SELECT Title, CommentCount, ViewCount, CommentsTotalScore, DisplayName, Reputation, Location
FROM (
        SELECT Posts.OwnerUserId, Posts.Title, Posts.CommentCount, Posts.ViewCount,
               CmtTotScr.CommentsTotalScore
        FROM (
                SELECT PostId, SUM(Score) AS CommentsTotalScore
                FROM Comments
                GROUP BY PostId
            ) AS CmtTotScr
        JOIN Posts ON Posts.Id = CmtTotScr.PostId
        WHERE Posts.PostTypeId=1
    ) AS PostsBestComments
JOIN Users ON PostsBestComments.OwnerUserId = Users.Id
ORDER BY CommentsTotalScore DESC
LIMIT 10
"""

def wykonaj_zapytanie(zap, k, conn_):
    """
    Wykonuje zapytanie zap (str) o numerze k (int) przy użyciu połączenia z bazą w obiekcie conn_.
    Wypisuje czas wykonania zapytania w sekundach (dokładnie tzw. Wall Clock Time, czyli czas rzeczywisty, który upłynął).
    Zwraca wynik zapytania jako ramkę danych Pandas.
    """
    t0 = time.time()  # zapisanie obecnego czasu do zmiennej
    wynik = pd.read_sql_query(zap, conn_)  # wykonanie zapytania
    t = time.time() - t0  # czas (w sekundach), jaki zajęło wykonania zapytania
    print(f'Wykonano zapytanie {k} w {t:.2f}s.')
    return wynik

# Poniższy blok with wykonuje wszystkie 5 zapytań;
# Wyniki umieszcza w zmiennych sql_i.
with sqlite3.connect(SCIEZKA_BAZY) as conn:
    sql_1 = wykonaj_zapytanie(zapytanie_1, 1, conn)
    sql_2 = wykonaj_zapytanie(zapytanie_2, 2, conn)
    sql_3 = wykonaj_zapytanie(zapytanie_3, 3, conn)
    sql_4 = wykonaj_zapytanie(zapytanie_4, 4, conn)
    sql_5 = wykonaj_zapytanie(zapytanie_5, 5, conn)

Wykonano zapytanie 1 w 0.04s.
Wykonano zapytanie 2 w 0.24s.
Wykonano zapytanie 3 w 507.40s.
Wykonano zapytanie 4 w 0.50s.


Moduł `pickle` to system serializacji (konwersji danych do postaci mogącej być zapisaną na dysku/przesłaną przez sieć) obiektów Pythona.
W przeciwieństwie do plików np `csv` można zapisywać niemal dowonly obiekt Pythona. Minusem jest niemożliwość przeczytania takiego obiektu inaczej, niż przez program napisany w Pythonie.

Poniższy kod zapisze wyniki zapytań (ramiki danych Pandas) do takich plików, aby można je potem było szybko odczytać.
**Nie trzeba z niego korzystać, jeżeli nie ma problemów z wykonaniem zapytań SQL.**

In [40]:
# Zapisanie każdej z ramek danych opisujących wyniki zapytań SQL do osobnego pliku pickle.
for i, df in enumerate([sql_1, sql_2, sql_3, sql_4, sql_5], 1):
    df.to_pickle(f'sql_{i}.pkl.gz')

In [41]:
# Wczytanie policzonych uprzednio wyników z plików pickle (możesz to zrobić, jeżeli zapytania wykonują się za długo).
sql_1, sql_2, sql_3, sql_4, sql_5= [
    pd.read_pickle(f'sql_{i}.pkl.gz') for i in range(1, 5 + 1)
]

## 3. Wyniki zapytań SQL odtworzone przy użyciu metod pakietu Pandas.

Wynikowa ramka danych do zapytania 1 popwinna nazwyać się `pandas_1`, do drugiego `pandas_2` itd.

### Zadanie 1

In [42]:
try:
    # tu umiesc swoje końcowe rozwiazanie
    # wynikowa ramka danych powinna się nazywać
    # pandas_1
    # ...
    # sprawdzenie równoważności wyników
    
    df=Users[["Location","UpVotes"]]

    df=df.loc[df.Location!="",:]

    df=df.groupby('Location').sum().rename(columns={"UpVotes":"TotalUpVotes"})\
                                                                    .sort_values("TotalUpVotes",ascending=False).head(10).reset_index()
    pandas_1=df
    
    print(pandas_1.equals(sql_1))

except Exception as e:
    print("Zad. 1: niepoprawny wynik.")
    print(e)

True


### Zadanie 2

In [43]:
try:
    # tu umiesc swoje końcowe rozwiazanie
    # wynikowa ramka danych powinna się nazywać
    # pandas_2
    # ...
    # sprawdzenie równoważności wyników
    
    df2=Posts.loc[(Posts.PostTypeId==1) | (Posts.PostTypeId==2),:]

    df2=df2[["CreationDate", "Score"]]

    df2['CreationDate']= pd.to_datetime(df2['CreationDate'])

    df2['Year']= df2['CreationDate'].dt.strftime('%Y')

    df2['Month']= df2['CreationDate'].dt.strftime('%m')

    df2= df2.groupby(["Year","Month"]).agg(MaxScore= ("Score","max"), PostsNumber=("Score","size"))

    df2=df2.loc[(df2.PostsNumber>1000),:]

    df2=df2.reset_index()

    df2= df2[["Year","Month","PostsNumber","MaxScore"]]

    pandas_2=df2
    
    print(pandas_2.equals(sql_2))

except Exception as e:
    print("Zad. 2: niepoprawny wynik.")
    print(e)

True


### Zadanie 3

In [44]:
try:
    # tu umiesc swoje końcowe rozwiazanie
    # wynikowa ramka danych powinna się nazywać
    # pandas_3
    # ...
    # sprawdzenie równoważności wyników
    
    df3=Posts.loc[(Posts.PostTypeId==1), ["OwnerUserId","ViewCount"]]
    
    df3= df3.groupby(["OwnerUserId"]).agg(TotalViews= ("ViewCount","sum"))
    
    Questions=df3

    merged = pd.merge(Questions, Users, left_on='OwnerUserId', right_on='Id', how='inner')
    
    looking_columns= merged[["Id","DisplayName","TotalViews"]].sort_values(["TotalViews"],ascending=False).reset_index()
    
    looking_final_data= looking_columns[["Id","DisplayName","TotalViews"]].head(10)
    
    pandas_3=looking_final_data
    
    print(pandas_3.equals(sql_3))

except Exception as e:
    print("Zad. 3: niepoprawny wynik.")
    print(e)

True


### Zadanie 4

In [91]:
try:
    
    Considering_Posts= Posts.loc[(Posts.PostTypeId==2),["OwnerUserId","Id"]]

    our_data= Considering_Posts.groupby(["OwnerUserId"]).agg(AnswersNumber= ("Id","size"))

    Answers=our_data
    Answers= Answers.loc[Answers.AnswersNumber.notnull(),:].reset_index()

    second_considering= Posts.loc[(Posts.PostTypeId==1),["OwnerUserId","Id"]]

    second_considering= second_considering.groupby(["OwnerUserId"]).agg(QuestionsNumber= ("OwnerUserId","size"))

    Questions= second_considering
    Questions= Questions.loc[Questions.QuestionsNumber.notnull(),:].reset_index()

    connected_data= pd.merge(Questions, Answers, left_on='OwnerUserId', right_on='OwnerUserId', how='inner')

    connected_data=connected_data.loc[(connected_data.AnswersNumber>connected_data.QuestionsNumber),:]\
                                                                    .sort_values(["AnswersNumber"],ascending=False)
    PostsCounts=connected_data
    PostsCounts=PostsCounts.head(5)

    looking_data= pd.merge(PostsCounts, Users, left_on='OwnerUserId', right_on='Id', how='inner')
    looking_data= looking_data[["DisplayName", "QuestionsNumber", "AnswersNumber", "Location", "Reputation","UpVotes", "DownVotes"]]
    looking_data = looking_data.replace(np.nan, None)

    pandas_4=looking_data
    
    print(pandas_4.equals(sql_4))

except Exception as e:
    print("Zad. 4: niepoprawny wynik.")
    print(e)

True


### Zadanie 5

In [ ]:
try:
    # tu umiesc swoje końcowe rozwiazanie
    # wynikowa ramka danych powinna się nazywać
    # pandas_5
    # ...
    # sprawdzenie równoważności wyników
    
    first_data= Comments.loc[["PostId","Score"]]
    first_considering= first_data.groupby(["PostId"]).agg(CommentsTotalScore= ("Score","sum"))
    CmtTotScr= first_considering

    connected_considering= pd.merge(CmtTotScr, Posts, left_on='PostId', right_on='Id', how='inner')

    PostsBestComments= connected_considering.loc[(connected_considering.PostTypeId==1),:]

    ultimate_considering= pd.merge(PostsBestComments, Users, left_on='OwnerUserId', right_on='Id', how='inner')\
                                                                .sort_values(["CommentsTotalScore"],ascending=False)
    ultimate_considering= ultimate_considering[["Title", "CommentCount", "ViewCount", "CommentsTotalScore", "DisplayName", "Reputation", "Location"]]
    ultimate_considering= ultimate_considering.head(10)

    pandas_5=ultimate_considering
    
    print(pandas_5.equals(sql_5))

except Exception as e:
    print("Zad. 5: niepoprawny wynik.")
    print(e)